# Day-wise Invoice Sum Report for Multiple Sheets
This notebook reads three Excel sheets, filters out unwanted rows, groups by day number, and writes the results to a new Excel file for easy analysis.

In [53]:
# Get necessary imports
import pandas as pd
import openpyxl
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment, numbers
from openpyxl.utils import get_column_letter
from datetime import datetime
import calendar
from copy import copy  # For copying Excel cell styles


## Define the sheets and paths to process

In [54]:
# Automated Path Configuration
import os
import glob
from datetime import datetime, timedelta
import calendar
import pandas as pd

def get_month_year_combinations(latest_month_year):
    """
    Get the latest month, last month, and last year combinations
    """
    # Parse the latest month-year
    month_name, year = latest_month_year.split('-')
    year = int(year)
    month_num = list(calendar.month_name).index(month_name)
    
    # Calculate last month
    if month_num == 1:  # January
        last_month_num = 12
        last_month_year_num = year - 1
    else:
        last_month_num = month_num - 1
        last_month_year_num = year
    
    # Calculate last year same month
    last_year_month_num = month_num
    last_year_year_num = year - 1
    
    # Convert back to names
    last_month_name = calendar.month_name[last_month_num]
    last_year_month_name = calendar.month_name[last_year_month_num]
    
    return {
        'latest': {'month': month_name, 'year': year, 'folder': f"{month_name}-{year}"},
        'last_month': {'month': last_month_name, 'year': last_month_year_num, 'folder': f"{last_month_name}-{last_month_year_num}"},
        'last_year': {'month': last_year_month_name, 'year': last_year_year_num, 'folder': f"{last_year_month_name}-{last_year_year_num}"}
    }

def find_file_by_keyword(folder_path, keyword):
    """
    Find a file in the folder that contains the keyword in its name
    """
    if not os.path.exists(folder_path):
        return None
    
    files = os.listdir(folder_path)
    for file in files:
        if keyword.lower() in file.lower() and file.endswith('.xlsx'):
            return os.path.join(folder_path, file)
    return None

def get_sheet_name_with_keyword(file_path, keyword):
    """
    Get the sheet name that contains the keyword
    """
    try:
        xl = pd.ExcelFile(file_path)
        for sheet_name in xl.sheet_names:
            if keyword.lower() in sheet_name.lower():
                return sheet_name
        # If no sheet with keyword found, return the first sheet
        return xl.sheet_names[0] if xl.sheet_names else None
    except:
        return None

def setup_automated_paths(latest_month_year, base_folder="DSR"):
    """
    Setup all paths automatically based on the latest month-year input
    
    Parameters:
    latest_month_year: str - Format: "June-2025"
    base_folder: str - Base folder name (default: "DSR")
    
    Returns:
    dict containing all the required paths and configurations
    """
    
    # Get month-year combinations
    dates = get_month_year_combinations(latest_month_year)
    
    # Base DSR folder path
    dsr_path = os.path.join(os.getcwd(), base_folder)
    
    # Prepare results
    sheet_info = []
    target_info = {}
    session_info = {}
    
    # Process each period (last_month, last_year, latest)
    periods = ['last_month', 'last_year', 'latest']
    display_names = [
        f"{dates['last_month']['month']} {dates['last_month']['year'] % 100}",  # May 25
        f"{dates['last_year']['month']} {dates['last_year']['year'] % 100}",   # June 24
        f"{dates['latest']['month']} {dates['latest']['year'] % 100}"          # June 25
    ]
    
    for i, period in enumerate(periods):
        period_data = dates[period]
        folder_path = os.path.join(dsr_path, period_data['folder'])
        
        # Find invoice file
        invoice_file = find_file_by_keyword(folder_path, 'invoice')
        if invoice_file:
            # Get the first sheet (since invoice files have only one sheet)
            try:
                xl = pd.ExcelFile(invoice_file)
                sheet_name = xl.sheet_names[0] if xl.sheet_names else 'Sheet1'
            except:
                sheet_name = 'Sheet1'
            
            # Make path relative to current working directory
            rel_path = os.path.relpath(invoice_file, os.getcwd())
            sheet_info.append((rel_path, sheet_name, display_names[i]))
    
    # Setup target information (using latest month)
    latest_folder = os.path.join(dsr_path, dates['latest']['folder'])
    target_file = find_file_by_keyword(latest_folder, 'target')
    if target_file:
        target_sheet = get_sheet_name_with_keyword(target_file, 'target')
        if not target_sheet:
            target_sheet = 'Target'  # Default if not found
        
        target_info = {
            'path': os.path.relpath(target_file, os.getcwd()),
            'sheet': target_sheet
        }
    
    # Setup session information (using latest month)
    traffic_file = find_file_by_keyword(latest_folder, 'traffic')
    if traffic_file:
        download_sheet = get_sheet_name_with_keyword(traffic_file, 'download')
        if not download_sheet:
            # If no download sheet found, get the first sheet
            try:
                xl = pd.ExcelFile(traffic_file)
                download_sheet = xl.sheet_names[0] if xl.sheet_names else 'Sheet1'
            except:
                download_sheet = 'Sheet1'
        
        session_info = {
            'path': os.path.relpath(traffic_file, os.getcwd()),
            'sheet': download_sheet
        }
    
    return {
        'sheet_info': sheet_info,
        'target_info': target_info,
        'session_info': session_info,
        'dates': dates
    }

# Define target Excel file 
output_path = 'invoice_day_channel_report_compatible.xlsx'

# AUTOMATED PATH CONFIGURATION
# Get user input for latest month-year and DSR path
latest_month_year = input("Enter the latest month-year (e.g., 'June-2025'): ").strip()
dsr_folder_path = input("Enter full path to DSR folder (or press Enter for default './DSR'): ").strip() or None

# Setup all paths automatically
try:
    config = setup_automated_paths(latest_month_year, dsr_folder_path)
    
    # Extract configuration
    sheet_info = config['sheet_info']
    target_config = config['target_info']
    session_config = config['session_info']
    
    print(f"\n✅ Configuration successful!")
    if dsr_folder_path:
        print(f"📁 Using DSR folder: {dsr_folder_path}")
    print(f"📁 Found {len(sheet_info)} invoice files:")
    for i, (path, sheet, display) in enumerate(sheet_info):
        print(f"   {i+1}. {display}: {path} -> {sheet}")
    
    if target_config:
        print(f"🎯 Target file: {target_config['path']} -> {target_config['sheet']}")
    else:
        print("⚠️  No target file found - using fallback")
    
    if session_config:
        print(f"📊 Session file: {session_config['path']} -> {session_config['sheet']}")
    else:
        print("⚠️  No session file found - using fallback")
    
except Exception as e:
    print(f"❌ Error in automated setup: {e}")
    print("🔄 Falling back to manual configuration...")
    
    # Fallback to manual configuration
    sheet_info = [
        ('test2/may25-final.xlsx', 'Sheet1', 'May 25'),   # Last month raw sheet
        ('test2/June24_Invoice.xlsx', 'Raw data June 24', 'June 24'),        # Last year raw sheet
        ('test2/June25.xlsx', 'Sheet1', 'June 25')                # Latest month raw sheet
    ]
    target_config = {'path': 'test2/Target_June_25.xlsx', 'sheet': 'Target-June25'}
    session_config = {'path': 'test2/June_2025_Daily traffic.xlsx', 'sheet': 'download - 2025-01-08T160122.10'}

# Constants for easier sheet reference - DO NOT USE THESE DIRECTLY
# Instead, use the index to get the specific dataframe
LAST_MONTH_INDEX = 0  # Last month index
LAST_YEAR_INDEX = 1   # Last year index  
LATEST_MONTH_INDEX = 2      # Latest month index

# Display names for column headers
LAST_MONTH_DISPLAY = sheet_info[LAST_MONTH_INDEX][2] if len(sheet_info) > LAST_MONTH_INDEX else "Last Month"
LAST_YEAR_DISPLAY = sheet_info[LAST_YEAR_INDEX][2] if len(sheet_info) > LAST_YEAR_INDEX else "Last Year"
LATEST_DISPLAY = sheet_info[LATEST_MONTH_INDEX][2] if len(sheet_info) > LATEST_MONTH_INDEX else "Latest"

# Target sheet information  
TARGET_PATH = target_config['path'] if target_config else 'test2/Target_June_25.xlsx'
TARGET_SHEET = target_config['sheet'] if target_config else 'Target-June25'

# Read the session data with specific columns
important_columns = [
    'Day',
    'Channel', 
    'Sessions',
    'Purchases',
    'Purchase revenue',
    'CG',
    'Category'
]

# Read the session data
session_file_path = session_config['path'] if session_config else "test2/June_2025_Daily traffic.xlsx"
session_sheet_name = session_config['sheet'] if session_config else "download - 2025-01-08T160122.10"

try:
    session_df = pd.read_excel(session_file_path, sheet_name=session_sheet_name)
    print(f"📈 Session data loaded successfully from: {session_file_path}")
except Exception as e:
    print(f"⚠️  Error loading session data: {e}")
    print("Please check the file path and sheet name manually.")

❌ Error in automated setup: join() argument must be str, bytes, or os.PathLike object, not 'NoneType'
🔄 Falling back to manual configuration...
📈 Session data loaded successfully from: test2/June_2025_Daily traffic.xlsx
📈 Session data loaded successfully from: test2/June_2025_Daily traffic.xlsx


In [55]:
# Define target Excel file 
output_path = 'invoice_day_channel_report_compatible.xlsx'

# AUTOMATED PATH CONFIGURATION
# Get user input for latest month-year
latest_month_year = latest_month_year.strip()

# Setup all paths automatically
try:
    config = setup_automated_paths(latest_month_year)
    
    # Extract configuration
    sheet_info = config['sheet_info']
    target_config = config['target_info']
    session_config = config['session_info']
    
    print(f"\n✅ Configuration successful!")
    print(f"📁 Found {len(sheet_info)} invoice files:")
    for i, (path, sheet, display) in enumerate(sheet_info):
        print(f"   {i+1}. {display}: {path} -> {sheet}")
    
    if target_config:
        print(f"🎯 Target file: {target_config['path']} -> {target_config['sheet']}")
    else:
        print("⚠️  No target file found - using fallback")
    
    if session_config:
        print(f"📊 Session file: {session_config['path']} -> {session_config['sheet']}")
    else:
        print("⚠️  No session file found - using fallback")
    
except Exception as e:
    print(f"❌ Error in automated setup: {e}")
    print("🔄 Falling back to manual configuration...")
    
    # Fallback to manual configuration
    sheet_info = [
        ('test2/may25-final.xlsx', 'Sheet1', 'May 25'),   # Last month raw sheet
        ('test2/June24_Invoice.xlsx', 'Raw data June 24', 'June 24'),        # Last year raw sheet
        ('test2/June25.xlsx', 'Sheet1', 'June 25')                # Latest month raw sheet
    ]
    target_config = {'path': 'test2/Target_June_25.xlsx', 'sheet': 'Target-June25'}
    session_config = {'path': 'test2/June_2025_Daily traffic.xlsx', 'sheet': 'download - 2025-01-08T160122.10'}

# Constants for easier sheet reference - DO NOT USE THESE DIRECTLY
# Instead, use the index to get the specific dataframe
LAST_MONTH_INDEX = 0  # Last month index
LAST_YEAR_INDEX = 1   # Last year index  
LATEST_MONTH_INDEX = 2      # Latest month index

# Display names for column headers
LAST_MONTH_DISPLAY = sheet_info[LAST_MONTH_INDEX][2] if len(sheet_info) > LAST_MONTH_INDEX else "Last Month"
LAST_YEAR_DISPLAY = sheet_info[LAST_YEAR_INDEX][2] if len(sheet_info) > LAST_YEAR_INDEX else "Last Year"
LATEST_DISPLAY = sheet_info[LATEST_MONTH_INDEX][2] if len(sheet_info) > LATEST_MONTH_INDEX else "Latest"

# Target sheet information  
TARGET_PATH = target_config['path'] if target_config else 'test2/Target_June_25.xlsx'
TARGET_SHEET = target_config['sheet'] if target_config else 'Target-June25'

# Read the session data with specific columns
important_columns = [
    'Day',
    'Channel', 
    'Sessions',
    'Purchases',
    'Purchase revenue',
    'CG',
    'Category'
]

# Read the session data
session_file_path = session_config['path'] if session_config else "test2/June_2025_Daily traffic.xlsx"
session_sheet_name = session_config['sheet'] if session_config else "download - 2025-01-08T160122.10"

try:
    session_df = pd.read_excel(session_file_path, sheet_name=session_sheet_name)
    print(f"📈 Session data loaded successfully from: {session_file_path}")
except Exception as e:
    print(f"⚠️  Error loading session data: {e}")
    print("Please check the file path and sheet name manually.")


✅ Configuration successful!
📁 Found 3 invoice files:
   1. May 25: DSR\May-2025\may25_Invoice.xlsx -> Sheet1
   2. June 24: DSR\June-2024\June24_Invoice.xlsx -> Raw data June 24
   3. June 25: DSR\June-2025\June25_Invoice.xlsx -> Sheet1
🎯 Target file: DSR\June-2025\Target_June_25.xlsx -> Target-June25
📊 Session file: DSR\June-2025\June_2025_Daily_traffic.xlsx -> download - 2025-01-08T160122.10
📈 Session data loaded successfully from: DSR\June-2025\June_2025_Daily_traffic.xlsx
📈 Session data loaded successfully from: DSR\June-2025\June_2025_Daily_traffic.xlsx


## Collect day-wise sums for each sheet

In [56]:
# Collect day-wise and TYPE-wise sums for each sheet
results = []
type_results = []
dfs = []  # Store the processed dataframes for each sheet

# First, process each sheet and store the dataframe, day sum, and type sum
for idx, (path, sheet, display_name) in enumerate(sheet_info):
    df = pd.read_excel(path, sheet_name=sheet)
    filtered_df = df[~df['idg'].isin(['FOC', 'Remove', 'WRT'])].copy()
    filtered_df['InvoiceDay'] = pd.to_datetime(filtered_df['InvoiceDate'], dayfirst=True, errors='coerce').dt.day
    
    # Map CC to Jumbo.ae in the TYPE column
    filtered_df['TYPE'] = filtered_df['TYPE'].replace('CC', 'Jumbo.ae')
    
    # Day-wise sum
    invoice_day_sum = filtered_df.groupby('InvoiceDay')['Amount Invoiced W.O. VAT'].sum()
    results.append((idx, invoice_day_sum))  # Store the index instead of sheet name
    
    # TYPE-wise sum for Jumbo.ae and EA
    filtered_type = filtered_df[filtered_df['TYPE'].isin(['Jumbo.ae', 'EA'])]
    sum_by_day_type = filtered_type.groupby(['InvoiceDay', 'TYPE'])['Amount Invoiced W.O. VAT'].sum().unstack(fill_value=0)
    type_results.append((idx, sum_by_day_type))  # Store the index instead of sheet name
    
    # Store the processed dataframe
    dfs.append(filtered_df)

type_results

[(0,
  TYPE                EA    Jumbo.ae
  InvoiceDay                        
  1            82528.440  131579.770
  2            72707.104  137289.165
  3           103473.880  101385.259
  4           168560.074  108395.948
  5            58094.577  113710.142
  6            58095.246   89224.399
  7            37775.141   96162.604
  8            61612.441  120124.135
  9            55628.192   89562.995
  10          134201.993  103498.824
  11           72807.087   60733.907
  12           66942.023  114640.947
  13          102211.760   91853.379
  14          112782.215   84981.141
  15           71914.388  120735.735
  16          116657.573  139414.822
  17           83215.021   69736.386
  18          109342.020   73419.887
  19          150136.517  111620.875
  20           53445.182   97200.961
  21           86518.866   87950.652
  22           87820.262  126864.630
  23           70848.688   87096.973
  24          100283.105   81711.063
  25          107369.686   91275.

## Combine results into a single DataFrame

In [57]:
# Get all days
all_days = set()
for _, s in results:
    all_days.update(s.index)
all_days = sorted(all_days)

# Convert results to a DataFrame using sheet indices
output = pd.DataFrame({'Day': all_days})
for idx, s in results:
    sheet_display = sheet_info[idx][2]  # Get display name from sheet_info
    output[sheet_display] = output['Day'].map(s)  # Use display name as column name

## Sum the 'Target' column by day number from the 'Date' column in the target sheet

## Calculate Percentage Differences
Add comparison columns for each channel:
- 'v/s Last Year': Percentage difference between first sheet (Raw data May 24) and latest sheet (Sheet1)
- 'v/s Last Month': Percentage difference between middle sheet (Raw data April 25) and latest sheet (Sheet1)
- 'v/s Target': Percentage difference between Target value and latest sheet (Sheet1)

In [58]:
# Get target sums by day and channel using constants
target_df = pd.read_excel(TARGET_PATH, sheet_name=TARGET_SHEET)
target_sums = target_df.groupby(['Date', 'Channel'])['Target'].sum().unstack(fill_value=0).round(6)

In [59]:
# Define channels and prepare data for final output (changed order to Jumbo.ae, EA, Total)
channels = ['Jumbo.ae', 'EA', 'Total']
# Use display names for column headers, but keep original indices for data processing
display_names = [display_name for _, _, display_name in sheet_info]
subcolumns = display_names[:2] + ['Target'] + [display_names[2]] + ['v/s Target', 'v/s Last Year', 'v/s Last Month']

# Create the MultiIndex
arrays = [[], []]
for channel in channels:
    for subcol in subcolumns:
        arrays[0].append(channel)
        arrays[1].append(subcol)

multiindex = pd.MultiIndex.from_arrays(arrays, names=['Channel', 'Type'])

# Process data by channel
multiindex_data = {}
for channel in channels:
    channel_data = {}
    
    # Process the raw data using indices for data access
    for idx, (sheet_idx, df) in enumerate(type_results):
        if channel == 'Total':
            channel_data[sheet_idx] = df.sum(axis=1)
        else:
            channel_data[sheet_idx] = df[channel]

    # Add target data
    if channel == 'Total':
        channel_data['Target'] = target_sums.sum(axis=1)
    else:
        channel_data['Target'] = target_sums[channel]

    # Calculate percentage differences using indices
    vs_last_year = []
    for day in all_days:
        latest_sheet_value = channel_data[LATEST_MONTH_INDEX].get(day, 0)
        first_sheet_value = channel_data[LAST_YEAR_INDEX].get(day, 0)
        
        if first_sheet_value == 0:
            if latest_sheet_value == 0:
                pct_diff = 0
            else:
                pct_diff = float('inf')
        else:
            pct_diff = ((latest_sheet_value - first_sheet_value) / first_sheet_value) * 100
            pct_diff = int(round(pct_diff))
        
        vs_last_year.append(pct_diff)
    
    vs_last_month = []
    for day in all_days:
        latest_sheet_value = channel_data[LATEST_MONTH_INDEX].get(day, 0)
        middle_sheet_value = channel_data[LAST_MONTH_INDEX].get(day, 0)
        
        if middle_sheet_value == 0:
            if latest_sheet_value == 0:
                pct_diff = 0
            else:
                pct_diff = float('inf')
        else:
            pct_diff = ((latest_sheet_value - middle_sheet_value) / middle_sheet_value) * 100
            pct_diff = int(round(pct_diff))
        
        vs_last_month.append(pct_diff)
    
    vs_target = []
    for day in all_days:
        latest_sheet_value = channel_data[LATEST_MONTH_INDEX].get(day, 0)
        target_value = channel_data['Target'].get(day, 0)
        
        if target_value == 0:
            if latest_sheet_value == 0:
                pct_diff = 0
            else:
                pct_diff = float('inf')
        else:
            pct_diff = ((latest_sheet_value ) / target_value) * 100
            pct_diff = int(round(pct_diff))
        
        vs_target.append(pct_diff)
    
    # Store all calculated columns
    channel_data['v/s Last Year'] = vs_last_year
    channel_data['v/s Last Month'] = vs_last_month
    channel_data['v/s Target'] = vs_target
    multiindex_data[channel] = channel_data

In [60]:
# Prepare data for DataFrame
all_data = []

# Create mapping from display names to sheet indices for data access
display_to_idx = {
    LAST_MONTH_DISPLAY: LAST_MONTH_INDEX,
    LAST_YEAR_DISPLAY: LAST_YEAR_INDEX,
    LATEST_DISPLAY: LATEST_MONTH_INDEX
}

for day in all_days:
    row = []
    for channel in channels:
        for subcol in subcolumns:
            if subcol in ['v/s Target', 'v/s Last Year', 'v/s Last Month']:
                # Get the index of this day in the list
                day_index = all_days.index(day)
                row.append(multiindex_data[channel][subcol][day_index])
            elif subcol == 'Target':
                # Target data remains the same
                row.append(multiindex_data[channel]['Target'].get(day, 0))
            else:
                # Map display name to sheet index for data access
                sheet_idx = display_to_idx.get(subcol)
                if sheet_idx is not None:
                    row.append(multiindex_data[channel][sheet_idx].get(day, 0))
                else:
                    row.append(0)  # Default if mapping not found
    all_data.append(row)

# Create final DataFrame
final_output = pd.DataFrame(all_data, columns=multiindex)
final_output.insert(0, 'Day', all_days)

# Add day names based on the first day of the month provided by user
# We'll add this in Excel formatting since we need the user input for first day of month

# Save initial data to Excel
final_output.to_excel(output_path)

In [61]:
# Format the Excel file
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment
from openpyxl.utils import get_column_letter
from openpyxl.formatting.rule import Rule
from openpyxl.styles.differential import DifferentialStyle
from datetime import datetime, timedelta

# Define the output path (make sure this file exists or create it first)
output_path = "invoice_day_channel_report_compatible.xlsx"

# Get the first day of the month from user
while True:
    first_day = input('Enter the first day of the month (Monday/Tuesday/Wednesday/Thursday/Friday/Saturday/Sunday): ').strip().lower()
    if first_day in ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']:
        break
    print('Invalid input! Please enter a valid day name.')

# Create a mapping of days to their position in a week (0=Monday to 6=Sunday)
day_positions = {
    'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3,
    'friday': 4, 'saturday': 5, 'sunday': 6
}
day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Get position of the first day (0-6, where 0 is Monday)
first_day_position = day_positions[first_day]

def get_week_info(day_of_month, first_day_pos):
    """Get week information for a given day.
    Returns (week_number, is_first_partial_week)"""
    # For days in the first partial week
    if first_day_pos > 0:  # If month doesn't start on Monday
        days_till_next_monday = 7 - first_day_pos
        if day_of_month <= days_till_next_monday:
            return 1, True
        # Adjust day number to calculate remaining weeks
        adjusted_day = day_of_month - days_till_next_monday
        return (adjusted_day - 1) // 7 + 2, False
    else:  # If month starts on Monday
        return (day_of_month - 1) // 7 + 1, False

def get_day_name(day_number, first_day_pos):
    """Get the day name for a given day of month"""
    # Calculate the day of week (0-6, where 0 is Monday)
    day_of_week = (first_day_pos + day_number - 1) % 7
    return day_names[day_of_week]

try:
    wb = load_workbook(output_path)
    ws = wb.active
except FileNotFoundError:
    print(f"Error: File {output_path} not found. Please ensure the file exists.")
    exit(1)

# Get dimensions
data_rows = ws.max_row
data_cols = ws.max_column

# Insert a new column for day names after the Day column
ws.insert_cols(3)  # Insert after Day column (column 2)

# Define styles
header_font = Font(bold=True, color="FFFFFF")
header_fill = PatternFill("solid", fgColor="4472C4")
subheader_fill = PatternFill("solid", fgColor="8EA9DB")
total_fill = PatternFill("solid", fgColor="D9E1F2")
border_style = Side(style='thin')
border = Border(left=border_style, right=border_style, top=border_style, bottom=border_style)

# Insert title row
ws.insert_rows(1)
title_cell = ws.cell(row=1, column=1)
title_cell.value = f"Invoice Day Channel Report - Generated on {datetime.now().strftime('%Y-%m-%d')}"
title_cell.font = Font(bold=True, size=14)
title_cell.alignment = Alignment(horizontal="center")

# Set up the day name column headers
ws.cell(row=2, column=3).value = ""  # Channel level header
ws.cell(row=3, column=3).value = "Day Name"  # Type level header

# Apply formatting to headers
for col in range(1, data_cols + 2):  # +2 to account for the new column and title row
    # Format the cell in title row
    ws.cell(row=1, column=col).border = border
    
    # Top header (channel)
    top_header_cell = ws.cell(row=2, column=col)
    top_header_cell.font = header_font
    top_header_cell.fill = header_fill
    top_header_cell.alignment = Alignment(horizontal="center", vertical="center")
    top_header_cell.border = border
    
    # Second header (sheet/target/comparison)
    second_header_cell = ws.cell(row=3, column=col)
    second_header_cell.font = header_font
    second_header_cell.fill = subheader_fill
    second_header_cell.alignment = Alignment(horizontal="center", vertical="center")
    second_header_cell.border = border

# Populate the day name column
day_col_idx = 2  # Day column (accounting for title row)
day_name_col_idx = 3  # Day Name column
data_start_row = 4  # First data row

# Add day names for each day number
for row in range(data_start_row, data_rows + 2):
    day_cell = ws.cell(row=row, column=day_col_idx)
    day_name_cell = ws.cell(row=row, column=day_name_col_idx)
    
    if isinstance(day_cell.value, (int, float)):
        # Get the day name for this day number
        day_name = get_day_name(int(day_cell.value), first_day_position)
        day_name_cell.value = day_name
        day_name_cell.alignment = Alignment(horizontal="center")
    
    # Apply border and fill based on row
    day_name_cell.border = border
    day_name_cell.fill = PatternFill("solid", fgColor="F2F2F2") if row % 2 == 0 else PatternFill()

# Auto-adjust column widths for all columns
for col in range(1, data_cols + 2):  # +2 to account for the new column and title row
    max_length = 0
    for row in range(1, data_rows + 2):
        cell_value = ws.cell(row=row, column=col).value
        if cell_value:
            max_length = max(max_length, len(str(cell_value)))
    adjusted_width = max(max_length + 2, 12)  # minimum width of 12
    ws.column_dimensions[get_column_letter(col)].width = adjusted_width

# Format data cells
for row in range(4, data_rows + 2):  # Start after headers and title
    # Apply row striping
    row_fill = PatternFill("solid", fgColor="F2F2F2") if row % 2 == 0 else PatternFill()
    
    for col in range(1, data_cols + 2):  # +2 to account for the new column and title row
        if col == day_name_col_idx:  # Skip day name column as it's already formatted
            continue
            
        cell = ws.cell(row=row, column=col)
        cell.fill = row_fill
        
        header_value = ws.cell(row=3, column=col).value
        
        # Format based on content
        if isinstance(cell.value, (int, float)):
            # Format percentage columns
            if header_value in ['v/s Target', 'v/s Last Year', 'v/s Last Month']:
                if cell.value == float('inf'):
                    cell.value = 'N/A'
                else:
                    value = int(round(cell.value)) # Raw percentage value, e.g., -10, 25, 150
                    
                    if header_value == 'v/s Target':
                        # For v/s Target, original text display was absolute value + %
                        cell.value = f"{abs(value)}%" 
                        # Coloring for v/s Target
                        if value >= 100:
                            cell.font = Font(color="006100")  # Dark green text
                            cell.fill = PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type='solid')  # Light green fill
                        else:
                            cell.font = Font(color="9C0006")  # Dark red text
                            cell.fill = PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid')  # Light red fill
                    else:  # For 'v/s Last Year' and 'v/s Last Month'
                        if value > 0:
                            cell.value = f"+{value}%"
                            cell.font = Font(color="006100")  # Dark green text
                            cell.fill = PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type='solid')  # Light green fill
                        elif value < 0:
                            cell.value = f"{value}%"  # Negative sign is included with 'value'
                            cell.font = Font(color="9C0006")  # Dark red text
                            cell.fill = PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid')  # Light red fill
                        else:  # value == 0
                            cell.value = "0%"
                            cell.font = Font(color="000000") # Black text for neutral
                            # cell.fill = PatternFill() # No specific fill, row striping applies, or set a neutral one
                cell.alignment = Alignment(horizontal="center")
            else: # Not a percentage column
                cell.number_format = '#,##0'
        
        cell.border = border
        
        # Center align Day column
        if header_value == 'Day':
            cell.alignment = Alignment(horizontal="center")

# Group days into weeks and add subtotals
data_start_row = 4  # First data row
week_ranges = []  # Store (start_row, end_row, week_num) for each week
subtotal_rows = []  # Store the row numbers of subtotals for grand total calculation

# First, collect all the day rows and their week numbers
day_rows = []
for row in range(data_start_row, data_rows + 2):
    day = ws.cell(row=row, column=day_col_idx).value
    if isinstance(day, (int, float)):
        # Get week number and whether it's part of the first partial week
        week_num, is_partial = get_week_info(int(day), first_day_position)
        day_rows.append((row, day, week_num, is_partial))

# Now organize them into week ranges
current_week = None
week_start_row = None
is_current_partial = False

for i, (row, day, week, is_partial) in enumerate(day_rows):
    if current_week != week:
        if current_week is not None:
            # End the previous week
            week_ranges.append((week_start_row, row - 1, current_week, is_current_partial))
        # Start a new week
        current_week = week
        week_start_row = row
        is_current_partial = is_partial
    
    # Handle the last week
    if i == len(day_rows) - 1:
        week_ranges.append((week_start_row, row, week, is_partial))

# Insert subtotal rows
rows_added = 0
for start_row, end_row, week_num, is_partial in week_ranges:
    # Adjusted for previously added subtotal rows
    adjusted_start = start_row + rows_added
    adjusted_end = end_row + rows_added
    
    # Insert the subtotal row
    ws.insert_rows(adjusted_end + 1)
    rows_added += 1
    subtotal_rows.append(adjusted_end + 1)  # Store the subtotal row number
    
    # Format subtotal row
    for col in range(1, data_cols + 2):  # +2 to account for the new column and title row
        subtotal_cell = ws.cell(row=adjusted_end + 1, column=col)
        subtotal_cell.font = Font(bold=True)
        subtotal_cell.fill = total_fill
        subtotal_cell.border = border
        
        if col == day_col_idx:
            if is_partial:
                days_in_partial = 7 - first_day_position
                subtitle = f"Week 1 (Partial: {days_in_partial} days)"
            else:
                subtitle = f"Week {week_num}"
            subtotal_cell.value = f"{subtitle} Subtotal"
            subtotal_cell.alignment = Alignment(horizontal="center")
        
        # Add Day Name subtotal cell
        elif col == day_name_col_idx:
            subtotal_cell.value = ""  # Leave empty for subtotals
        
        elif col > day_name_col_idx:  # Adjust for the Day Name column
            header_value = ws.cell(row=3, column=col).value
            col_letter = get_column_letter(col)
            
            if header_value == 'v/s Target':
                # Get the column letter for "Test" (current month) for this channel - now at column F
                sheet1_col = col - 1  # Current month data (Test) is 1 column before v/s Target
                sheet1_letter = get_column_letter(sheet1_col)
                
                # Get the column letter for "Target" for this channel - now at column E
                target_col = col - 2  # Target data is 2 columns before v/s Target
                target_letter = get_column_letter(target_col)
                
                # Create formula to calculate percentage: (Test) / Target
                # Ensure the formula outputs a number, and handle division by zero by outputting 0.
                formula = f"=IF({target_letter}{adjusted_end+1}=0,0,ROUND(({sheet1_letter}{adjusted_end+1})/{target_letter}{adjusted_end+1}*100,0))"
                subtotal_cell.value = formula
                subtotal_cell.number_format = '0\"%\"'  # Apply percentage number format
                subtotal_cell.alignment = Alignment(horizontal="center")
            
            elif header_value == 'v/s Last Year':
                # Get the column letter for "Test" (current month) for this channel - now at column F
                sheet1_col = col - 2  # Current month data (Test) is 2 columns before v/s Last Year
                sheet1_letter = get_column_letter(sheet1_col)
                
                # Get the column letter for "Raw data May 24" (last year) for this channel - at column D
                may24_col = col - 4  # Last year data (Raw data May 24) is 4 columns before v/s Last Year
                may24_letter = get_column_letter(may24_col)
                
                # Create formula to calculate percentage: (Test - Raw data May 24) / Raw data May 24
                # Ensure the formula outputs a number, and handle division by zero by outputting 0.
                formula = f"=IF({may24_letter}{adjusted_end+1}=0,0,ROUND(({sheet1_letter}{adjusted_end+1}-{may24_letter}{adjusted_end+1})/{may24_letter}{adjusted_end+1}*100,0))"
                subtotal_cell.value = formula
                subtotal_cell.number_format = '0\"%\"' # Display as percentage
                subtotal_cell.alignment = Alignment(horizontal="center")
                
            elif header_value == 'v/s Last Month':
                # Get the column letter for "Test" (current month) for this channel - now at column F
                sheet1_col = col - 3  # Current month data (Test) is 3 columns before v/s Last Month
                sheet1_letter = get_column_letter(sheet1_col)
                
                # Get the column letter for "Raw data April 25" (last month) for this channel - at column C
                april25_col = col - 6  # Last month data (Raw data April 25) is 6 columns before v/s Last Month
                april25_letter = get_column_letter(april25_col)
                
                # Create formula to calculate percentage: (Test - Raw data April 25) / Raw data April 25
                # Ensure the formula outputs a number, and handle division by zero by outputting 0 for the numeric part.
                # The original formula appended "%" making it text.
                formula_numeric_part = f"IF({april25_letter}{adjusted_end+1}=0,0,ROUND(({sheet1_letter}{adjusted_end+1}-{april25_letter}{adjusted_end+1})/{april25_letter}{adjusted_end+1}*100,0))"
                
                # The rule for these columns is > 0 is green, < 0 is red.
                # The original code for data rows adds a "+" sign.
                # For consistency with conditional formatting rules (which expect numbers), we'll keep it numeric.
                # The display format will be handled by number_format if needed, or Excel's default for numbers.
                # However, the original code for subtotal formulas for these columns also appended "%".
                # Let's make them numeric and apply number format for consistency with 'v/s Target' approach.
                formula = f"=IF({april25_letter}{adjusted_end+1}=0,0,ROUND(({sheet1_letter}{adjusted_end+1}-{april25_letter}{adjusted_end+1})/{april25_letter}{adjusted_end+1}*100,0))"
                subtotal_cell.value = formula
                subtotal_cell.number_format = '0\"%\"' # Display as percentage
                subtotal_cell.alignment = Alignment(horizontal="center")
                
            elif header_value not in ['v/s Target', 'v/s Last Year', 'v/s Last Month']:
                # Calculate sum for this week's range
                subtotal_cell.value = f"=SUM({col_letter}{adjusted_start}:{col_letter}{adjusted_end})"
                subtotal_cell.number_format = '#,##0'

# Add grand total row
grand_total_row = data_rows + rows_added + 2
ws.insert_rows(grand_total_row)

# Format grand total row
for col in range(1, data_cols + 2):  # +2 to account for the new column and title row
    grand_total_cell = ws.cell(row=grand_total_row, column=col)
    grand_total_cell.font = Font(bold=True)
    grand_total_cell.fill = total_fill
    grand_total_cell.border = border
    
    if col == day_col_idx:
        grand_total_cell.value = "Grand Total"
        grand_total_cell.alignment = Alignment(horizontal="center")
    elif col == day_name_col_idx:
        grand_total_cell.value = ""
    elif col > day_name_col_idx:  # Adjust for the Day Name column
        header_value = ws.cell(row=3, column=col).value
        col_letter = get_column_letter(col)
        
        if header_value == 'v/s Target':
            # Get the column letters for Test (column F) and Target (column E) for grand total
            sheet1_col = col - 1
            sheet1_letter = get_column_letter(sheet1_col)
            target_col = col - 2
            target_letter = get_column_letter(target_col)
            
            # Create formula for grand total percentage - numeric output
            formula = f"=IF({target_letter}{grand_total_row}=0,0,ROUND(({sheet1_letter}{grand_total_row})/{target_letter}{grand_total_row}*100,0))"
            grand_total_cell.value = formula
            grand_total_cell.number_format = '0\"%\"'  # Apply percentage number format
            grand_total_cell.alignment = Alignment(horizontal="center")
            
        elif header_value == 'v/s Last Year':
            # Get the column letters for Test (column F) and Raw data May 24 (column D) for grand total
            sheet1_col = col - 2
            sheet1_letter = get_column_letter(sheet1_col)
            may24_col = col - 4
            may24_letter = get_column_letter(may24_col)
            
            # Create formula for grand total percentage - numeric output
            formula = f"=IF({may24_letter}{grand_total_row}=0,0,ROUND(({sheet1_letter}{grand_total_row}-{may24_letter}{grand_total_row})/{may24_letter}{grand_total_row}*100,0))"
            grand_total_cell.value = formula
            grand_total_cell.number_format = '0\"%\"' # Display as percentage
            grand_total_cell.alignment = Alignment(horizontal="center")
            
        elif header_value == 'v/s Last Month':
            # Get the column letters for Test (column F) and Raw data April 25 (column C) for grand total
            sheet1_col = col - 3
            sheet1_letter = get_column_letter(sheet1_col)
            april25_col = col - 6
            april25_letter = get_column_letter(april25_col)
            
            # Create formula for grand total percentage - numeric output
            formula = f"=IF({april25_letter}{grand_total_row}=0,0,ROUND(({sheet1_letter}{grand_total_row}-{april25_letter}{grand_total_row})/{april25_letter}{grand_total_row}*100,0))"
            grand_total_cell.value = formula
            grand_total_cell.number_format = '0\"%\"' # Display as percentage
            grand_total_cell.alignment = Alignment(horizontal="center")
            
        elif header_value not in ['v/s Target', 'v/s Last Year', 'v/s Last Month']:
            # Build formula to sum only the weekly subtotal rows
            if subtotal_rows:  # Only if we have subtotal rows
                subtotal_ranges = [f"{col_letter}{row}" for row in subtotal_rows]
                formula = "=SUM(" + ",".join(subtotal_ranges) + ")"
                grand_total_cell.value = formula
                grand_total_cell.number_format = '#,##0'

# Apply conditional formatting with simpler rules
# Define styles for conditional formatting
green_fill = PatternFill(start_color='C6EFCE', end_color='C6EFCE', fill_type='solid')
red_fill = PatternFill(start_color='FFC7CE', end_color='FFC7CE', fill_type='solid')
green_font = Font(color='006100', bold=True)
red_font = Font(color='9C0006', bold=True)

# Create differential styles
green_style = DifferentialStyle(fill=green_fill, font=green_font)
red_style = DifferentialStyle(fill=red_fill, font=red_font)

# Apply conditional formatting to percentage columns in subtotal and grand total rows
percentage_columns = []
for col in range(day_name_col_idx + 1, data_cols + 2):
    header_value = ws.cell(row=3, column=col).value
    if header_value in ['v/s Target', 'v/s Last Year', 'v/s Last Month']:
        percentage_columns.append(col)

# Apply formatting to subtotal rows
for subtotal_row in subtotal_rows:
    for col in percentage_columns:
        header_value = ws.cell(row=3, column=col).value
        cell_range = f"{get_column_letter(col)}{subtotal_row}"
        
        try:
            if header_value == 'v/s Target':
                # For Target: Green if >= 100, Red if < 100
                green_rule = Rule(type="cellIs", operator="greaterThanOrEqual", formula=[100], dxf=green_style)
                red_rule = Rule(type="cellIs", operator="lessThan", formula=[100], dxf=red_style)
            else:
                # For Last Year and Last Month: Green if > 0, Red if < 0
                green_rule = Rule(type="cellIs", operator="greaterThan", formula=[0], dxf=green_style)
                red_rule = Rule(type="cellIs", operator="lessThan", formula=[0], dxf=red_style)
            
            ws.conditional_formatting.add(cell_range, green_rule)
            ws.conditional_formatting.add(cell_range, red_rule)
        except Exception as e:
            print(f"Warning: Could not apply conditional formatting to {cell_range}: {e}")

# Apply formatting to grand total row
for col in percentage_columns:
    header_value = ws.cell(row=3, column=col).value
    cell_range = f"{get_column_letter(col)}{grand_total_row}"
    
    try:
        if header_value == 'v/s Target':
            # For Target: Green if >= 100, Red if < 100
            green_rule = Rule(type="cellIs", operator="greaterThanOrEqual", formula=[100], dxf=green_style)
            red_rule = Rule(type="cellIs", operator="lessThan", formula=[100], dxf=red_style)
        else:
            # For Last Year and Last Month: Green if > 0, Red if < 0
            green_rule = Rule(type="cellIs", operator="greaterThan", formula=[0], dxf=green_style)
            red_rule = Rule(type="cellIs", operator="lessThan", formula=[0], dxf=red_style)
        
        ws.conditional_formatting.add(cell_range, green_rule)
        ws.conditional_formatting.add(cell_range, red_rule)
    except Exception as e:
        print(f"Warning: Could not apply conditional formatting to {cell_range}: {e}")

# Manually merge headers - be careful for future changes
try:
    # Only merge if we have enough columns
    if data_cols >= 24:  # Adjust based on your actual column count
        ws.merge_cells('D2:J2')
        ws.merge_cells('K2:Q2')
        ws.merge_cells('R2:X2')
    else:
        print("Warning: Not enough columns to merge headers as specified")
        ws.merge_cells('D2:J2')
        ws.merge_cells('K2:Q2')
        ws.merge_cells('R2:X2')
except Exception as e:
    print(f"Warning: Could not merge headers: {e}")

# Save the final formatted workbook
try:
    wb.save(output_path)
    print(f"Created final Excel report at {output_path}")
    print(f"First day of month was {first_day.capitalize()}, weeks are aligned to start on Monday")
    print(f"Added day name column next to the Day column for better readability")
except Exception as e:
    print(f"Error saving file: {e}")
    # Try saving with a different name
    backup_path = output_path.replace('.xlsx', '_backup.xlsx')
    try:
        wb.save(backup_path)
        print(f"Saved backup file as {backup_path}")
    except Exception as e2:
        print(f"Could not save backup file either: {e2}")

Created final Excel report at invoice_day_channel_report_compatible.xlsx
First day of month was Sunday, weeks are aligned to start on Monday
Added day name column next to the Day column for better readability


## Session Data Processing
Read and display the data from session.xlsx file

In [62]:


# Apply filters with case-insensitive comparison for Category
# 1. Remove Gift Card from Category (case-insensitive)
# 2. Remove EA and Endless Aisle from CG
session_df = session_df[
    (~session_df['Category'].str.lower().str.contains('gift card', na=False)) & 
    (~session_df['CG'].isin(['EA', 'Endless Aisle']))
]

session_df['Date'] = pd.to_datetime(session_df['Date'], format='%Y%m%d', errors='coerce')

# Convert 'Day' column to day number (handles both date and string types)
session_df['Day'] = session_df['Date'].dt.day

# Select only the important columns
session_df = session_df[important_columns]

session_df


,Day,Channel,Sessions,Purchases,Purchase revenue,CG,Category
0,1.0,Paid Perf,1.0,1.0,2498.999999,NaN,NaN
1,1.0,Organic,1.0,1.0,3899.000000,NaN,NaN
2,1.0,Paid Perf,1.0,1.0,8999.000001,Online,Television & Home Theaters
3,1.0,Paid Perf,1.0,1.0,1699.000000,Online,Headphones & Speakers
4,1.0,Email,1.0,1.0,498.000000,Online,Health and Personal Care
...,...,...,...,...,...,...,...
9550,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9551,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9552,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9553,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
month_days = 31

## Channel-wise Session, Purchases, and Purchase Revenue
Group the filtered session data by Channel and aggregate Sessions, Purchases, and Purchase revenue for Email, Organic, Paid Perf, and Paid Other.

In [64]:
# Import IPython display to avoid conflicts with overridden display variable
from IPython.display import display

# Get all unique channels
channels_of_interest = sorted(session_df['Channel'].dropna().unique())

# Filter for the channels
filtered = session_df[session_df['Channel'].isin(channels_of_interest)]

# Group by Channel and aggregate
agg = filtered.groupby('Channel')[['Sessions', 'Purchases', 'Purchase revenue']].sum().reset_index()

# Display the result
print("Available channels:")
print(channels_of_interest)
print("\nAggregated data by Channel (Sessions, Purchases, Purchase revenue):")
display(agg)

Available channels:
['Affiliates', 'Display', 'Email', 'Organic', 'Paid Other', 'Paid Perf', 'Paid Social', 'Referral']

Aggregated data by Channel (Sessions, Purchases, Purchase revenue):


,Channel,Sessions,Purchases,Purchase revenue
0,Affiliates,7775.0,39.0,88463.999992
1,Display,524.0,0.0,0.000000
2,Email,8961.0,11.0,15899.000005
3,Organic,218339.0,151.0,312597.600021
4,Paid Other,1085.0,0.0,0.000000
5,Paid Perf,92401.0,173.0,279033.499994
6,Paid Social,185.0,0.0,0.000000
7,Referral,43.0,1.0,999.000000


## Day and Channel-wise Session, Purchases, and Purchase Revenue
Group the filtered session data by Day number and Channel, aggregating Sessions, Purchases, and Purchase revenue for Email, Organic, Paid Perf, and Paid Other.

In [65]:
# Group by Day and Channel, aggregate Sessions, Purchases, and Purchase revenue
agg_day_channel = filtered.groupby(['Day', 'Channel'])[['Sessions', 'Purchases', 'Purchase revenue']].sum().reset_index()

# Display the result
print("Aggregated data by Day and Channel (Sessions, Purchases, Purchase revenue):")
display(agg_day_channel)

Aggregated data by Day and Channel (Sessions, Purchases, Purchase revenue):


,Day,Channel,Sessions,Purchases,Purchase revenue
0,1.0,Affiliates,717.0,1.0,1199.000000
1,1.0,Display,28.0,0.0,0.000000
2,1.0,Email,1513.0,5.0,4938.000004
3,1.0,Organic,17785.0,13.0,20482.000002
4,1.0,Paid Other,143.0,0.0,0.000000
...,...,...,...,...,...
78,12.0,Email,558.0,0.0,0.000000
79,12.0,Organic,18917.0,9.0,17954.000000
80,12.0,Paid Perf,6431.0,13.0,30605.900006
81,12.0,Paid Social,2.0,0.0,0.000000


## Pivot Table: Day-wise Channel Split for Sessions, Purchases, and Purchase Revenue
A table with super columns for Sessions, Purchases, and Purchase revenue, each split by channel (Email, Organic, Paid Perf, Paid Other), and Day as the index.

In [66]:
import numpy as np
import pandas as pd

# Assuming 'filtered' DataFrame is already defined in the environment

# Create a complete DataFrame with all days (1-31) for each channel
all_days = list(range(1, 32))  # All days in a month (1-31)
all_channels = sorted(filtered['Channel'].unique())

# Create empty dataframe with all possible day-channel combinations
full_month_data = []
for day in all_days:
    for channel in all_channels:
        full_month_data.append({
            'Day': day,
            'Channel': channel,
            'Sessions': 0,
            'Purchases': 0,
            'Purchase revenue': 0.0
        })

# Create full month DataFrame
full_month_df = pd.DataFrame(full_month_data)

# Update with actual data where available
actual_data = filtered.groupby(['Day', 'Channel']).agg({
    'Sessions': 'sum',
    'Purchases': 'sum',
    'Purchase revenue': 'sum'
}).reset_index()

# Merge actual data with full month data
full_month_df = pd.merge(
    full_month_df,
    actual_data,
    on=['Day', 'Channel'],
    how='left',
    suffixes=('_full', '')
).fillna(0)

# Keep only the columns we need
full_month_df = full_month_df[['Day', 'Channel', 'Sessions', 'Purchases', 'Purchase revenue']]

# Create the pivot table using the full month data
pivot = full_month_df.pivot_table(
    index='Day',
    columns='Channel',
    values=['Sessions', 'Purchases', 'Purchase revenue'],
    aggfunc='sum',
    fill_value=0
)

# Get all channels in sorted order
channels_order = sorted(full_month_df['Channel'].unique())
metrics_order = ['Sessions', 'Purchases', 'Purchase revenue']

# Build MultiIndex columns in the desired order
pivot = pivot.reindex(columns=pd.MultiIndex.from_product([metrics_order, channels_order]))

# Calculate CVR and AOV for each channel
cvr_data = {}
aov_data = {}

for channel in channels_order:
    # Calculate CVR (Conversion Rate) = (Purchases / Sessions) * 100
    cvr = (pivot[('Purchases', channel)] / pivot[('Sessions', channel)] * 100).round(2)
    cvr_data[channel] = cvr.replace([np.inf, -np.inf], 0)  # Handle division by zero
    
    # Calculate AOV (Average Order Value) = Purchase revenue / Purchases
    aov = (pivot[('Purchase revenue', channel)] / pivot[('Purchases', channel)]).round(2)
    aov_data[channel] = aov.replace([np.inf, -np.inf], 0)  # Handle division by zero

# Add CVR and AOV to the pivot table
for channel in channels_order:
    pivot[('CVR', channel)] = cvr_data[channel]
    pivot[('AOV', channel)] = aov_data[channel]

# Update metrics order to include new columns
metrics_order = ['Sessions', 'Purchases', 'Purchase revenue', 'CVR', 'AOV']

# Reorder all columns according to the updated metrics
pivot = pivot.reindex(columns=pd.MultiIndex.from_product([metrics_order, channels_order]))

# Reset index for display
pivot = pivot.reset_index()

# Display the result
print("Day-wise channel split with Sessions, Purchases, Purchase revenue, CVR, and AOV (all days 1-31):")
display(pivot)

Day-wise channel split with Sessions, Purchases, Purchase revenue, CVR, and AOV (all days 1-31):


Day   Sessions                                                            \
       Affiliates Display   Email  Organic Paid Other Paid Perf Paid Social   
0    1      717.0    28.0  1513.0  17785.0      143.0    8699.0         3.0   
1    2      750.0    31.0  1591.0  17498.0       44.0    8386.0         0.0   
2    3      711.0    39.0   735.0  18318.0       34.0    7807.0         6.0   
3    4      630.0    39.0   262.0  18834.0       32.0    7901.0        35.0   
4    5      552.0    80.0   558.0  17143.0       12.0    7989.0        49.0   
5    6      555.0   100.0   830.0  21662.0       76.0    8083.0        34.0   
6    7      507.0    93.0   942.0  16903.0      106.0    8270.0        29.0   
7    8      572.0    85.0  1033.0  16817.0      506.0    8262.0        14.0   
8    9      563.0     9.0   441.0  18020.0       72.0    7038.0         4.0   
9   10      663.0     4.0   187.0  18733.0       32.0    6719.0         7.0   
10  11      779.0    10.0   311.0  17709.0       28.0    6816.0         2.0   
11  12      776.0     6.0   558.0  18917.0        0.0    6431.0         2.0   
12  13        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
13  14        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
14  15        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
15  16        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
16  17        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
17  18        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
18  19        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
19  20        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
20  21        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
21  22        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
22  23        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
23  24        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
24  25        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
25  26        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
26  27        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
27  28        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
28  29        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
29  30        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
30  31        0.0     0.0     0.0      0.0        0.0       0.0         0.0   

             Purchases  ...         CVR                 AOV                   \
   Referral Affiliates  ... Paid Social Referral Affiliates Display    Email   
0       0.0        1.0  ...         0.0      NaN    1199.00     NaN   987.60   
1       0.0        6.0  ...         NaN      NaN    2065.67     NaN      NaN   
2       0.0        2.0  ...         0.0      NaN    3648.50     NaN      NaN   
3       0.0        7.0  ...         0.0      NaN    2212.43     NaN      NaN   
4       0.0        6.0  ...         0.0      NaN    1912.33     NaN  2098.33   
5       0.0        2.0  ...         0.0      NaN    1499.00     NaN      NaN   
6       0.0        1.0  ...         0.0      NaN     479.00     NaN  3598.00   
7       0.0        3.0  ...         0.0      NaN    3009.00     NaN   499.00   
8       0.0        4.0  ...         0.0      NaN    3324.00     NaN   569.00   
9       0.0        2.0  ...         0.0      NaN    2474.00     NaN      NaN   
10      0.0        2.0  ...         0.0      NaN    2084.00     NaN      NaN   
11     43.0        3.0  ...         0.0     2.33    1899.00     NaN      NaN   
12      0.0        0.0  ...         NaN      NaN        NaN     NaN      NaN   
13      0.0        0.0  ...         NaN      NaN        NaN     NaN      NaN   
14      0.0        0.0  ...         NaN      NaN        NaN     NaN      NaN   
15      0.0        0.0  ...        

In [67]:
import numpy as np
import pandas as pd

# Apply styling to the pivot table
def style_df(val, props=''):
    return props

# Create a complete DataFrame with all days for each channel
all_days = list(range(1, month_days + 1))  # Use month_days instead of hardcoding 31
all_channels = sorted(filtered['Channel'].unique())

# Create empty dataframe with all possible day-channel combinations
full_month_data = []
for day in all_days:
    for channel in all_channels:
        full_month_data.append({
            'Day': day,
            'Channel': channel,
            'Sessions': 0,
            'Purchases': 0,
            'Purchase revenue': 0.0
        })

# Create full month DataFrame
full_month_df = pd.DataFrame(full_month_data)

# Update with actual data where available
actual_data = filtered.groupby(['Day', 'Channel']).agg({
    'Sessions': 'sum',
    'Purchases': 'sum',
    'Purchase revenue': 'sum'
}).reset_index()

# Merge actual data with full month data
full_month_df = pd.merge(
    full_month_df,
    actual_data,
    on=['Day', 'Channel'],
    how='left',
    suffixes=('_full', '')
).fillna(0)

# Keep only the columns we need
full_month_df = full_month_df[['Day', 'Channel', 'Sessions', 'Purchases', 'Purchase revenue']]

# Create the pivot table using the full month data
pivot = full_month_df.pivot_table(
    index='Day',
    columns='Channel',
    values=['Sessions', 'Purchases', 'Purchase revenue'],
    aggfunc='sum',
    fill_value=0
)

# Get all channels in sorted order
channels_order = sorted(full_month_df['Channel'].unique())
metrics_order = ['Sessions', 'Purchases', 'Purchase revenue']

# Build MultiIndex columns in the desired order
pivot = pivot.reindex(columns=pd.MultiIndex.from_product([metrics_order, channels_order]))

# Calculate CVR and AOV for each channel
cvr_data = {}
aov_data = {}

for channel in channels_order:
    # Calculate CVR (Conversion Rate) = (Purchases / Sessions) * 100
    cvr = (pivot[('Purchases', channel)] / pivot[('Sessions', channel)] * 100).round(2)
    cvr_data[channel] = cvr.replace([np.inf, -np.inf], 0)  # Handle division by zero
    
    # Calculate AOV (Average Order Value) = Purchase revenue / Purchases
    aov = (pivot[('Purchase revenue', channel)] / pivot[('Purchases', channel)]).round(2)
    aov_data[channel] = aov.replace([np.inf, -np.inf], 0)  # Handle division by zero

# Add CVR and AOV to the pivot table
for channel in channels_order:
    pivot[('CVR', channel)] = cvr_data[channel]
    pivot[('AOV', channel)] = aov_data[channel]

# Update metrics order to include new columns
metrics_order = ['Sessions', 'Purchases', 'Purchase revenue', 'CVR', 'AOV']

# Reorder all columns according to the updated metrics
pivot = pivot.reindex(columns=pd.MultiIndex.from_product([metrics_order, channels_order]))

# Reset index for display
pivot = pivot.reset_index()

# Display the result
print(f"Day-wise channel split with Sessions, Purchases, Purchase revenue, CVR, and AOV (all days 1-{month_days}):")
display(pivot)

# Define the styling for different metrics
styled_pivot = pivot.style\
    .format({('CVR', channel): '{:.2f}%' for channel in channels_order})\
    .format({('AOV', channel): '${:,.2f}' for channel in channels_order})\
    .format({('Purchase revenue', channel): '${:,.2f}' for channel in channels_order})\
    .format({('Sessions', channel): '{:,.0f}' for channel in channels_order})\
    .format({('Purchases', channel): '{:,.0f}' for channel in channels_order})

# Display the styled pivot table
display(styled_pivot)

Day-wise channel split with Sessions, Purchases, Purchase revenue, CVR, and AOV (all days 1-31):


Day   Sessions                                                            \
       Affiliates Display   Email  Organic Paid Other Paid Perf Paid Social   
0    1      717.0    28.0  1513.0  17785.0      143.0    8699.0         3.0   
1    2      750.0    31.0  1591.0  17498.0       44.0    8386.0         0.0   
2    3      711.0    39.0   735.0  18318.0       34.0    7807.0         6.0   
3    4      630.0    39.0   262.0  18834.0       32.0    7901.0        35.0   
4    5      552.0    80.0   558.0  17143.0       12.0    7989.0        49.0   
5    6      555.0   100.0   830.0  21662.0       76.0    8083.0        34.0   
6    7      507.0    93.0   942.0  16903.0      106.0    8270.0        29.0   
7    8      572.0    85.0  1033.0  16817.0      506.0    8262.0        14.0   
8    9      563.0     9.0   441.0  18020.0       72.0    7038.0         4.0   
9   10      663.0     4.0   187.0  18733.0       32.0    6719.0         7.0   
10  11      779.0    10.0   311.0  17709.0       28.0    6816.0         2.0   
11  12      776.0     6.0   558.0  18917.0        0.0    6431.0         2.0   
12  13        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
13  14        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
14  15        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
15  16        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
16  17        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
17  18        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
18  19        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
19  20        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
20  21        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
21  22        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
22  23        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
23  24        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
24  25        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
25  26        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
26  27        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
27  28        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
28  29        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
29  30        0.0     0.0     0.0      0.0        0.0       0.0         0.0   
30  31        0.0     0.0     0.0      0.0        0.0       0.0         0.0   

             Purchases  ...         CVR                 AOV                   \
   Referral Affiliates  ... Paid Social Referral Affiliates Display    Email   
0       0.0        1.0  ...         0.0      NaN    1199.00     NaN   987.60   
1       0.0        6.0  ...         NaN      NaN    2065.67     NaN      NaN   
2       0.0        2.0  ...         0.0      NaN    3648.50     NaN      NaN   
3       0.0        7.0  ...         0.0      NaN    2212.43     NaN      NaN   
4       0.0        6.0  ...         0.0      NaN    1912.33     NaN  2098.33   
5       0.0        2.0  ...         0.0      NaN    1499.00     NaN      NaN   
6       0.0        1.0  ...         0.0      NaN     479.00     NaN  3598.00   
7       0.0        3.0  ...         0.0      NaN    3009.00     NaN   499.00   
8       0.0        4.0  ...         0.0      NaN    3324.00     NaN   569.00   
9       0.0        2.0  ...         0.0      NaN    2474.00     NaN      NaN   
10      0.0        2.0  ...         0.0      NaN    2084.00     NaN      NaN   
11     43.0        3.0  ...         0.0     2.33    1899.00     NaN      NaN   
12      0.0        0.0  ...         NaN      NaN        NaN     NaN      NaN   
13      0.0        0.0  ...         NaN      NaN        NaN     NaN      NaN   
14      0.0        0.0  ...         NaN      NaN        NaN     NaN      NaN   
15      0.0        0.0  ...        

In [68]:
# Export pivot table to Excel with formatting
output_path = 'session_channel_report.xlsx'
pivot.to_excel(output_path)

# Format the Excel file
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Border, Side, Alignment
from openpyxl.utils import get_column_letter

# # Get the first day of the month from user (reuse the same input)
# while True:
#     first_day = input('Enter the first day of the month (Monday/Tuesday/Wednesday/Thursday/Friday/Saturday/Sunday): ').strip().lower()
#     if first_day in ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']:
#         break
#     print('Invalid input! Please enter a valid day name.')

# # Create a mapping of days to their position in a week (0=Monday to 6=Sunday)
# day_positions = {
#     'monday': 0, 'tuesday': 1, 'wednesday': 2, 'thursday': 3,
#     'friday': 4, 'saturday': 5, 'sunday': 6
# }
# day_names = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# # Get position of the first day (0-6, where 0 is Monday)
# first_day_position = day_positions[first_day]


def get_week_info(day_of_month, first_day_pos):
    """Get week information for a given day.
    Returns (week_number, is_first_partial_week)"""
    if first_day_pos > 0:  # If month doesn't start on Monday
        days_till_next_monday = 7 - first_day_pos
        if day_of_month <= days_till_next_monday:
            return 1, True
        adjusted_day = day_of_month - days_till_next_monday
        return (adjusted_day - 1) // 7 + 2, False
    else:  # If month starts on Monday
        return (day_of_month - 1) // 7 + 1, False

def get_day_name(day_number, first_day_pos):
    """Get the day name for a given day of month"""
    day_of_week = (first_day_pos + day_number - 1) % 7
    return day_names[day_of_week]

wb = load_workbook(output_path)
ws = wb.active

# Get dimensions
data_rows = ws.max_row
data_cols = ws.max_column

# Insert a new column for day names after the Day column
ws.insert_cols(3)

# Define styles
header_font = Font(bold=True, color="FFFFFF")
header_fill = PatternFill("solid", fgColor="4472C4")
subheader_fill = PatternFill("solid", fgColor="8EA9DB")
total_fill = PatternFill("solid", fgColor="D9E1F2")
border_style = Side(style='thin')
border = Border(left=border_style, right=border_style, top=border_style, bottom=border_style)

# Insert title row
ws.insert_rows(1)
title_cell = ws.cell(row=1, column=1)
title_cell.value = f"Session Channel Report - Generated on {datetime.now().strftime('%Y-%m-%d')}"
title_cell.font = Font(bold=True, size=14)
title_cell.alignment = Alignment(horizontal="center")

# Set up headers
for col in range(1, data_cols + 2):
    # Format cells in title row
    ws.cell(row=1, column=col).border = border
    
    # Top header (metrics)
    top_header_cell = ws.cell(row=2, column=col)
    top_header_cell.font = header_font
    top_header_cell.fill = header_fill
    top_header_cell.border = border
    top_header_cell.alignment = Alignment(horizontal="center")
    
    # Second header (channels)
    second_header_cell = ws.cell(row=3, column=col)
    second_header_cell.font = header_font
    second_header_cell.fill = subheader_fill
    second_header_cell.border = border
    second_header_cell.alignment = Alignment(horizontal="center")

# Add day names
day_col_idx = 2
day_name_col_idx = 3
data_start_row = 4

# Add day names for each day number
for row in range(data_start_row, data_rows + 2):
    day_cell = ws.cell(row=row, column=day_col_idx)
    day_name_cell = ws.cell(row=row, column=day_name_col_idx)
    
    if isinstance(day_cell.value, (int, float)):
        day_name = get_day_name(int(day_cell.value), first_day_position)
        day_name_cell.value = day_name
        day_name_cell.alignment = Alignment(horizontal="center")
    
    day_name_cell.border = border
    day_name_cell.fill = PatternFill("solid", fgColor="F2F2F2") if row % 2 == 0 else PatternFill()

# Collect day rows and group into weeks
day_rows = []
for row in range(data_start_row, data_rows + 2):
    day = ws.cell(row=row, column=day_col_idx).value
    if isinstance(day, (int, float)):
        week_num, is_partial = get_week_info(int(day), first_day_position)
        day_rows.append((row, day, week_num, is_partial))

# Organize into week ranges
week_ranges = []
subtotal_rows = []
current_week = None
week_start_row = None
is_current_partial = False

for i, (row, day, week, is_partial) in enumerate(day_rows):
    if current_week != week:
        if current_week is not None:
            # End the previous week
            week_ranges.append((week_start_row, row - 1, current_week, is_current_partial))
        # Start a new week
        current_week = week
        week_start_row = row
        is_current_partial = is_partial
    
    # Handle the last week
    if i == len(day_rows) - 1:
        week_ranges.append((week_start_row, row, week, is_partial))

# Insert subtotal rows
rows_added = 0
for start_row, end_row, week_num, is_partial in week_ranges:
    adjusted_start = start_row + rows_added
    adjusted_end = end_row + rows_added
    
    ws.insert_rows(adjusted_end + 1)
    rows_added += 1
    subtotal_rows.append(adjusted_end + 1)
    
    # Format subtotal row
    for col in range(1, data_cols + 2):
        subtotal_cell = ws.cell(row=adjusted_end + 1, column=col)
        subtotal_cell.font = Font(bold=True)
        subtotal_cell.fill = total_fill
        subtotal_cell.border = border
        
        if col == day_col_idx:
            if is_partial:
                days_in_partial = 7 - first_day_position
                subtitle = f"Week 1 (Partial: {days_in_partial} days)"
            else:
                subtitle = f"Week {week_num}"
            subtotal_cell.value = f"{subtitle} Subtotal"
            subtotal_cell.alignment = Alignment(horizontal="center")
        
        # Add Day Name subtotal cell
        elif col == day_name_col_idx:
            subtotal_cell.value = ""  # Leave empty for subtotals
        
        elif col > day_name_col_idx:  # Adjust for the Day Name column
            header_value = ws.cell(row=3, column=col).value
            col_letter = get_column_letter(col)
            
            if header_value == 'v/s Target':
                # Get the column letter for "Test" (current month) for this channel - now at column F
                sheet1_col = col - 1  # Current month data (Test) is 1 column before v/s Target
                sheet1_letter = get_column_letter(sheet1_col)
                
                # Get the column letter for "Target" for this channel - now at column E
                target_col = col - 2  # Target data is 2 columns before v/s Target
                target_letter = get_column_letter(target_col)
                
                # Create formula to calculate percentage: (Test) / Target
                # Ensure the formula outputs a number, and handle division by zero by outputting 0.
                formula = f"=IF({target_letter}{adjusted_end+1}=0,0,ROUND(({sheet1_letter}{adjusted_end+1})/{target_letter}{adjusted_end+1}*100,0))"
                subtotal_cell.value = formula
                subtotal_cell.number_format = '0\"%\"'  # Apply percentage number format
                subtotal_cell.alignment = Alignment(horizontal="center")
            
            elif header_value == 'v/s Last Year':
                # Get the column letter for "Test" (current month) for this channel - now at column F
                sheet1_col = col - 2  # Current month data (Test) is 2 columns before v/s Last Year
                sheet1_letter = get_column_letter(sheet1_col)
                
                # Get the column letter for "Raw data May 24" (last year) for this channel - at column D
                may24_col = col - 4  # Last year data (Raw data May 24) is 4 columns before v/s Last Year
                may24_letter = get_column_letter(may24_col)
                
                # Create formula to calculate percentage: (Test - Raw data May 24) / Raw data May 24
                # Ensure the formula outputs a number, and handle division by zero by outputting 0.
                formula = f"=IF({may24_letter}{adjusted_end+1}=0,0,ROUND(({sheet1_letter}{adjusted_end+1}-{may24_letter}{adjusted_end+1})/{may24_letter}{adjusted_end+1}*100,0))"
                subtotal_cell.value = formula
                subtotal_cell.number_format = '0\"%\"' # Display as percentage
                subtotal_cell.alignment = Alignment(horizontal="center")
                
            elif header_value == 'v/s Last Month':
                # Get the column letter for "Test" (current month) for this channel - now at column F
                sheet1_col = col - 3  # Current month data (Test) is 3 columns before v/s Last Month
                sheet1_letter = get_column_letter(sheet1_col)
                
                # Get the column letter for "Raw data April 25" (last month) for this channel - at column C
                april25_col = col - 6  # Last month data (Raw data April 25) is 6 columns before v/s Last Month
                april25_letter = get_column_letter(april25_col)
                
                # Create formula to calculate percentage: (Test - Raw data April 25) / Raw data April 25
                # Ensure the formula outputs a number, and handle division by zero by outputting 0 for the numeric part.
                # The original formula appended "%" making it text.
                formula_numeric_part = f"IF({april25_letter}{adjusted_end+1}=0,0,ROUND(({sheet1_letter}{adjusted_end+1}-{april25_letter}{adjusted_end+1})/{april25_letter}{adjusted_end+1}*100,0))"
                
                # The rule for these columns is > 0 is green, < 0 is red.
                # The original code for data rows adds a "+" sign.
                # For consistency with conditional formatting rules (which expect numbers), we'll keep it numeric.
                # The display format will be handled by number_format if needed, or Excel's default for numbers.
                # However, the original code for subtotal formulas for these columns also appended "%".
                # Let's make them numeric and apply number format for consistency with 'v/s Target' approach.
                formula = f"=IF({april25_letter}{adjusted_end+1}=0,0,ROUND(({sheet1_letter}{adjusted_end+1}-{april25_letter}{adjusted_end+1})/{april25_letter}{adjusted_end+1}*100,0))"
                subtotal_cell.value = formula
                subtotal_cell.number_format = '0\"%\"' # Display as percentage
                subtotal_cell.alignment = Alignment(horizontal="center")
                
            elif header_value not in ['v/s Target', 'v/s Last Year', 'v/s Last Month']:
                # Calculate sum for this week's range
                subtotal_cell.value = f"=SUM({col_letter}{adjusted_start}:{col_letter}{adjusted_end})"
                subtotal_cell.number_format = '#,##0'

# Add grand total row
grand_total_row = data_rows + rows_added + 2
ws.insert_rows(grand_total_row)

# Format grand total row
for col in range(1, data_cols + 2):  # +2 to account for the new column and title row
    grand_total_cell = ws.cell(row=grand_total_row, column=col)
    grand_total_cell.font = Font(bold=True)
    grand_total_cell.fill = total_fill
    grand_total_cell.border = border
    
    if col == day_col_idx:
        grand_total_cell.value = "Grand Total"
        grand_total_cell.alignment = Alignment(horizontal="center")
    elif col == day_name_col_idx:
        grand_total_cell.value = ""
    elif col > day_name_col_idx:  # Adjust for the Day Name column
        header_value = ws.cell(row=3, column=col).value
        col_letter = get_column_letter(col)
        
        if header_value == 'v/s Target':
            # Get the column letters for Test (column F) and Target (column E) for grand total
            sheet1_col = col - 1
            sheet1_letter = get_column_letter(sheet1_col)
            target_col = col - 2
            target_letter = get_column_letter(target_col)
            
            # Create formula for grand total percentage - numeric output
            formula = f"=IF({target_letter}{grand_total_row}=0,0,ROUND(({sheet1_letter}{grand_total_row})/{target_letter}{grand_total_row}*100,0))"
            grand_total_cell.value = formula
            grand_total_cell.number_format = '0\"%\"'  # Apply percentage number format
            grand_total_cell.alignment = Alignment(horizontal="center")
            
        elif header_value == 'v/s Last Year':
            # Get the column letters for Test (column F) and Raw data May 24 (column D) for grand total
            sheet1_col = col - 2
            sheet1_letter = get_column_letter(sheet1_col)
            may24_col = col - 4
            may24_letter = get_column_letter(may24_col)
            
            # Create formula for grand total percentage - numeric output
            formula = f"=IF({may24_letter}{grand_total_row}=0,0,ROUND(({sheet1_letter}{grand_total_row}-{may24_letter}{grand_total_row})/{may24_letter}{grand_total_row}*100,0))"
            grand_total_cell.value = formula
            grand_total_cell.number_format = '0\"%\"' # Display as percentage
            grand_total_cell.alignment = Alignment(horizontal="center")
            
        elif header_value == 'v/s Last Month':
            # Get the column letters for Test (column F) and Raw data April 25 (column C) for grand total
            sheet1_col = col - 3
            sheet1_letter = get_column_letter(sheet1_col)
            april25_col = col - 6
            april25_letter = get_column_letter(april25_col)
            
            # Create formula for grand total percentage - numeric output
            formula = f"=IF({april25_letter}{grand_total_row}=0,0,ROUND(({sheet1_letter}{grand_total_row}-{april25_letter}{grand_total_row})/{april25_letter}{grand_total_row}*100,0))"
            grand_total_cell.value = formula
            grand_total_cell.number_format = '0\"%\"' # Display as percentage
            grand_total_cell.alignment = Alignment(horizontal="center")
            
        elif header_value not in ['v/s Target', 'v/s Last Year', 'v/s Last Month']:
            # Build formula to sum only the weekly subtotal rows
            if subtotal_rows:  # Only if we have subtotal rows
                subtotal_ranges = [f"{col_letter}{row}" for row in subtotal_rows]
                formula = "=SUM(" + ",".join(subtotal_ranges) + ")"
                grand_total_cell.value = formula
                grand_total_cell.number_format = '#,##0'

# Format numbers
for row in range(4, grand_total_row + 1):
    for col in range(4, data_cols + 2):
        cell = ws.cell(row=row, column=col)
        header = ws.cell(row=3, column=col).value
        
        if header == 'CVR':
            cell.number_format = '0.00%'  # Fixed escape sequence
        elif header == 'AOV' or header == 'Purchase revenue':
            cell.number_format = '$#,##0.00'  # Fixed escape sequence
        elif header in ['Sessions', 'Purchases']:
            cell.number_format = '#,##0'

# Auto-adjust column widths
for col in range(1, data_cols + 2):
    max_length = 0
    for row in range(1, grand_total_row + 1):
        cell_value = ws.cell(row=row, column=col).value
        if cell_value:
            max_length = max(max_length, len(str(cell_value)))
    adjusted_width = max(max_length + 2, 12)
    ws.column_dimensions[get_column_letter(col)].width = adjusted_width

# Calculate column spans for each metric
metrics = ['Sessions', 'Purchases', 'Purchase revenue', 'CVR', 'AOV']
channels_count = len(channels_order)
start_col = 4  # Start after Day and Day Name columns

for metric in metrics:
    end_col = start_col + channels_count - 1
    
    # Set the value in the first cell
    cell = ws.cell(row=2, column=start_col)
    cell.value = metric
    cell.font = header_font
    cell.fill = header_fill
    cell.alignment = Alignment(horizontal="center")
    
    # Then merge the cells
    ws.merge_cells(start_row=2, start_column=start_col, end_row=2, end_column=end_col)
    
    # Move to next section
    start_col = end_col + 1

# Save the workbook
wb.save(output_path)
print(f"Created session channel report at {output_path}")
print(f"First day of month was {first_day.capitalize()}, weeks are aligned to start on Monday")


Created session channel report at session_channel_report.xlsx
First day of month was Sunday, weeks are aligned to start on Monday


## Copy Session Channel Data
Copy data from session channel report to invoice day channel report, preserving all formatting

In [69]:
from openpyxl import load_workbook
from copy import copy

# Copy data from session_channel_report.xlsx to invoice_day_channel_report_compatible.xlsx
def copy_session_data():
    """
    Copy session data from session_channel_report.xlsx to invoice_day_channel_report_compatible.xlsx
    Starting from cell D2 in source to cell Y2 in destination
    """
    source_path = 'session_channel_report.xlsx'
    dest_path = 'invoice_day_channel_report_compatible.xlsx'
    
    # Load both workbooks
    source_wb = load_workbook(source_path,data_only=True)
    dest_wb = load_workbook(dest_path)
    
    source_ws = source_wb.active
    dest_ws = dest_wb.active
    
    # Get the range of data to copy (starts from D2)
    source_max_row = source_ws.max_row
    source_max_col = source_ws.max_column
    
    # Copy data from D2 onwards in source to Y2 onwards in destination
    source_start_col = 4  # Column D
    dest_start_col = 25  # Column Y
    
    # Copy values and formatting
    for row in range(2, source_max_row + 1):
        for col_offset in range(source_max_col - source_start_col + 1):
            source_col = source_start_col + col_offset
            dest_col = dest_start_col + col_offset
            
            # Get source cell
            source_cell = source_ws.cell(row=row, column=source_col)
            
            # Get destination cell
            dest_cell = dest_ws.cell(row=row, column=dest_col)
            
            # Copy value
            dest_cell.value = source_cell.value
            
            # Copy formatting
            if source_cell.has_style:
                dest_cell.font = copy(source_cell.font)
                dest_cell.fill = copy(source_cell.fill)
                dest_cell.border = copy(source_cell.border)
                dest_cell.alignment = copy(source_cell.alignment)
                dest_cell.number_format = source_cell.number_format
    
    # Save the destination workbook
    dest_wb.save(dest_path)
    print(f"Successfully copied session data from {source_path} to {dest_path}")

# Execute the copy function
copy_session_data()

Successfully copied session data from session_channel_report.xlsx to invoice_day_channel_report_compatible.xlsx


## Copy Session Channel Data with Merged Cells
Copy data from session channel report to invoice day channel report, preserving all formatting including merged cells

In [70]:
import os
def copy_session_data():
    """
    Copy session data from session_channel_report.xlsx to invoice_day_channel_report_compatible.xlsx
    Starting from cell D2 in source to cell Y2 in destination, preserving merged cells
    """
    source_path = 'session_channel_report.xlsx'
    dest_path = 'invoice_day_channel_report_compatible.xlsx'

    if os.path.exists(source_path):
        os.startfile(source_path)
    
    _=input("Open the session_channel_report.xlsx, save it, then close it again and press Enter to continue...")

    # Load both workbooks
    source_wb = load_workbook(source_path, data_only=True)
    dest_wb = load_workbook(dest_path)
    
    source_ws = source_wb.active
    dest_ws = dest_wb.active
    
    # Get the range of data to copy (starts from D2)
    source_max_row = source_ws.max_row
    source_max_col = source_ws.max_column
    
    # Copy data from D2 onwards in source to Y2 onwards in destination
    source_start_col = 4  # Column D
    dest_start_col = 25  # Column Y
    
    # First, handle any merged cells in the header rows
    # Get all merged ranges from source
    merged_ranges = source_ws.merged_cells.ranges
    
    # For each merged range in source that starts with our copied columns
    for merged_range in merged_ranges:
        if merged_range.min_col >= source_start_col:
            # Calculate the offset for the destination
            col_offset = dest_start_col - source_start_col
            
            # Create new merge range coordinates for destination
            new_min_col = merged_range.min_col + col_offset
            new_max_col = merged_range.max_col + col_offset
            new_range = f"{get_column_letter(new_min_col)}{merged_range.min_row}:{get_column_letter(new_max_col)}{merged_range.max_row}"
            
            # Merge the cells in destination
            try:
                dest_ws.merge_cells(new_range)
                
                # Copy the value from the top-left cell of the merge range
                source_value = source_ws.cell(merged_range.min_row, merged_range.min_col).value
                dest_ws.cell(merged_range.min_row, new_min_col).value = source_value
                
                # Copy formatting from the first cell of merge range
                source_cell = source_ws.cell(merged_range.min_row, merged_range.min_col)
                dest_cell = dest_ws.cell(merged_range.min_row, new_min_col)
                
                if source_cell.has_style:
                    dest_cell.font = copy(source_cell.font)
                    dest_cell.fill = copy(source_cell.fill)
                    dest_cell.border = copy(source_cell.border)
                    dest_cell.alignment = copy(source_cell.alignment)
                    dest_cell.number_format = source_cell.number_format
            except ValueError:
                # If merge range already exists, just update the value and formatting
                dest_ws.cell(merged_range.min_row, new_min_col).value = source_value
    
    # Now copy the rest of the data and formatting
    for row in range(2, source_max_row + 1):
        for col_offset in range(source_max_col - source_start_col + 1):
            source_col = source_start_col + col_offset
            dest_col = dest_start_col + col_offset
            
            # Skip if this cell is part of a merged range
            if any(merged_range.min_row <= row <= merged_range.max_row and 
                  merged_range.min_col <= source_col <= merged_range.max_col 
                  for merged_range in merged_ranges):
                continue
            
            # Get source cell
            source_cell = source_ws.cell(row=row, column=source_col)
            
            # Get destination cell
            dest_cell = dest_ws.cell(row=row, column=dest_col)
            
            # Copy value
            dest_cell.value = source_cell.value
            
            # Copy formatting if it has any
            if source_cell.has_style:
                dest_cell.font = copy(source_cell.font)
                dest_cell.fill = copy(source_cell.fill)
                dest_cell.border = copy(source_cell.border)
                dest_cell.alignment = copy(source_cell.alignment)
                dest_cell.number_format = source_cell.number_format
    
    # Save the destination workbook
    dest_wb.save(dest_path)
    print(f"Successfully copied session data from {source_path} to {dest_path}")

# Execute the copy function
copy_session_data()

Successfully copied session data from session_channel_report.xlsx to invoice_day_channel_report_compatible.xlsx


## Final Processing
Delete session_channel_report.xlsx and hide the first column in invoice_day_channel_report_compatible.xlsx

In [71]:
import os
from openpyxl import load_workbook

# Step 1: Delete session_channel_report.xlsx
if os.path.exists('session_channel_report.xlsx'):
    os.remove('session_channel_report.xlsx')
    print("Deleted 'session_channel_report.xlsx'")
else:
    print("'session_channel_report.xlsx' not found, skipping deletion")

# Step 2: Hide the first column in invoice_day_channel_report_compatible.xlsx
report_path = 'invoice_day_channel_report_compatible.xlsx'

# Load the workbook
wb = load_workbook(report_path)
ws = wb.active

# Hide the first column (column A)
ws.column_dimensions['A'].hidden = True
ws.freeze_panes = 'D4'  # Freeze the first three rows and the first three columns

# Save the workbook
wb.save(report_path)
print(f"Hidden the first column in '{report_path}'")
print("All processing complete!")

if os.path.exists('invoice_day_channel_report_compatible.xlsx'):
    os.startfile('invoice_day_channel_report_compatible.xlsx')

Deleted 'session_channel_report.xlsx'
Hidden the first column in 'invoice_day_channel_report_compatible.xlsx'
All processing complete!
